In [10]:
import tensorflow as tf
import numpy as np

In [11]:
x = tf.random.normal([1,4,4,1], 0, 1, tf.float32)


print(x.shape)

y = tf.keras.layers.MaxPooling2D((2, 2))(x)
y2 = tf.keras.layers.MaxPooling2D(2, 2)(x)

print(y)
print(y2)

(1, 4, 4, 1)
tf.Tensor(
[[[[1.8174407 ]
   [0.98337185]]

  [[0.78887016]
   [0.79055876]]]], shape=(1, 2, 2, 1), dtype=float32)
tf.Tensor(
[[[[1.8174407 ]
   [0.98337185]]

  [[0.78887016]
   [0.79055876]]]], shape=(1, 2, 2, 1), dtype=float32)


In [14]:
import tensorflow as tf
import numpy as np
from layers import ReshapeTensor, ExpandDimensions

inputs = tf.random.normal([10,8,8,2048])

depth = 256
atrous_rates = (1,2,3)
kernel_size = 3
stride = 1
rate = 1

depth_padding = 'same'

inputs_size = tf.shape(inputs)[1:3]

conv_1x1 = tf.keras.layers.Conv2D(depth, 1, strides=1, padding=depth_padding, dilation_rate=1, use_bias = False)(inputs)
conv_3x3_1 = tf.keras.layers.Conv2D(depth, 3, strides=1, padding=depth_padding, dilation_rate=atrous_rates[0], use_bias = False)(inputs)
conv_3x3_2 = tf.keras.layers.Conv2D(depth, 3, strides=1, padding=depth_padding, dilation_rate=atrous_rates[1], use_bias = False)(inputs)
conv_3x3_3 = tf.keras.layers.Conv2D(depth, 3, strides=1, padding=depth_padding, dilation_rate=atrous_rates[2], use_bias = False)(inputs)

print(np.shape(inputs))
# global average pooling


image_level_features = tf.keras.layers.GlobalAveragePooling2D(keepdims=True)(inputs)
print(np.shape(image_level_features))

image_level_features = tf.keras.layers.Conv2D(depth, 1, strides=1)(image_level_features)
        
#bilinearly upsample features
image_level_features = ReshapeTensor(inputs_size)(image_level_features)

print(np.shape(image_level_features))

net = tf.keras.layers.Concatenate(axis=3)([conv_1x1, conv_3x3_1, conv_3x3_2, conv_3x3_3, image_level_features])        
print(net.shape)
net = tf.keras.layers.Conv2D(depth, 1, strides=1)(net)
print(net.shape)
 

(10, 8, 8, 2048)
(10, 1, 1, 2048)
(10, 8, 8, 256)
(10, 8, 8, 1280)
(10, 8, 8, 256)


In [4]:
logits = [[4.0, 2.0, 1.0], [0.0, 5.0, 1.0]]
labels = [[1.0, 0.0, 0.0], [0.0, 0.8, 0.2]]
result = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits))

print(result)

tf.Tensor(0.4972955, shape=(), dtype=float32)


In [9]:
logits = [[4.0, 2.0, 1.0], [0.0, 5.0, 1.0]]
labels = [[1.0, 0.0, 0.0], [0.0, 0.8, 0.2]]
result = tf.keras.losses.CategoricalCrossentropy(from_logits=True)(labels,logits)

print(result)

tf.Tensor(0.4972955, shape=(), dtype=float32)


In [21]:
x = np.random.random_sample((1, 2, 2))
print(x.shape)
print(x)
print(np.shape(x)[1])

#y = np.reshape(x,(2,2))
#print(y.shape)
#print(y)

x1 = np.concatenate(x, axis=0)
print(x1.shape)
print(x1)

(1, 2, 2)
[[[0.15929904 0.79884429]
  [0.48919627 0.44281906]]]
2
(2, 2)
[[0.15929904 0.79884429]
 [0.48919627 0.44281906]]


In [94]:
from Networks import *
from layers import GradientReversalLayer

inputs = tf.random.normal(shape=(32, 8, 8, 256))

fc = Domain_Regressor_FullyConnected(1024,2)

x,logits = fc(inputs)

print(tf.shape(x))
print(tf.shape(logits))

#print(inputs.shape[3])

num_filters = inputs.shape[3]

print("#######################################")

#256, 128, 64
#for i in range(3): 
#    print(i)    
#    print(num_filters/(2**i))

conv = Domain_Regressor_Convolutional(2)


conv.summary()

grad = GradientReversalLayer()([inputs,0.037])

print(tf.shape(grad))

x1,logits1 = conv(grad)

print(tf.shape(x1))
print(tf.shape(logits1)[3])

#print(logits1)


tf.Tensor([32  2], shape=(2,), dtype=int32)
tf.Tensor([32  2], shape=(2,), dtype=int32)
#######################################


ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [93]:
from Networks import *
from layers import GradientReversalLayer

inputs = tf.random.normal(shape=(2, 4, 4, 2))

print(inputs)

grad = GradientReversalLayer()([inputs,0.037])

print(grad)

tf.Tensor(
[[[[ 0.70936286  0.19106193]
   [-0.14148995 -0.86487156]
   [ 0.00436472 -0.24014476]
   [ 0.30503157 -1.294186  ]]

  [[-0.963309   -0.25598574]
   [ 0.73387724 -1.9983513 ]
   [-0.92183936  0.43104553]
   [ 0.07676414  0.00497397]]

  [[ 2.1945407  -0.00639255]
   [ 0.33226424  1.7936802 ]
   [-0.8062077   0.24771525]
   [ 0.29570708  1.2209939 ]]

  [[ 0.1743099  -0.3744971 ]
   [-0.32156888  0.6944017 ]
   [-0.40737057 -1.2838    ]
   [ 0.7491835   0.9780987 ]]]


 [[[-1.2058235  -0.36863866]
   [ 1.8411677  -0.1519867 ]
   [ 0.20310535 -0.06209443]
   [ 0.2263793  -1.6121606 ]]

  [[ 0.11472436  1.4253123 ]
   [ 1.1035818   0.4909932 ]
   [ 0.91452783  0.08028387]
   [ 0.46495062  0.24182716]]

  [[-1.9747303  -1.0934762 ]
   [-1.7341897   0.65822434]
   [-0.17610936 -1.7028635 ]
   [-0.9721943   0.30758366]]

  [[-1.1341666   1.3776942 ]
   [ 0.38876748  0.20358185]
   [ 0.6469661   1.6811141 ]
   [-1.304598    1.8566558 ]]]], shape=(2, 4, 4, 2), dtype=float32)
tf.Ten

In [112]:
from tensorflow import keras

inputs = keras.Input(shape=(4, 4, 3))
processed = keras.layers.RandomCrop(width=32, height=32)(inputs)
conv = keras.layers.Conv2D(filters=2, kernel_size=3)(processed)
pooling = keras.layers.GlobalAveragePooling2D()(conv)
feature = keras.layers.Dense(10)(pooling)

full_model = keras.Model(inputs, feature)
backbone = keras.Model(processed, conv)
activations = keras.Model(conv, feature)
logits = keras.Model(inputs, [feature,pooling])

#print(pooling.shape)
print(logits.layers[-1].output.shape)



(None, 10)


In [90]:
for e in range(100):
    p = float(e) / 100
    print("Percentage of epochs: " + str(p))
    warmup = 1
    if e >= warmup:
        l = 2. / (1. + np.exp(-2.5 * p)) - 1
    else:
        l = 0
    print("lambda_p: " + str(l))

Percentage of epochs: 0.0
lambda_p: 0
Percentage of epochs: 0.01
lambda_p: 0.012499348999020832
Percentage of epochs: 0.02
lambda_p: 0.024994792968420665
Percentage of epochs: 0.03
lambda_p: 0.037482431757070334
Percentage of epochs: 0.04
lambda_p: 0.049958374957880025
Percentage of epochs: 0.05
lambda_p: 0.06241874674751258
Percentage of epochs: 0.06
lambda_p: 0.0748596906874992
Percentage of epochs: 0.07
lambda_p: 0.08727737447415773
Percentage of epochs: 0.08
lambda_p: 0.0996679946249559
Percentage of epochs: 0.09
lambda_p: 0.11202778108923983
Percentage of epochs: 0.1
lambda_p: 0.12435300177159614
Percentage of epochs: 0.11
lambda_p: 0.13663996695649616
Percentage of epochs: 0.12
lambda_p: 0.14888503362331806
Percentage of epochs: 0.13
lambda_p: 0.1610846096413192
Percentage of epochs: 0.14
lambda_p: 0.17323515783466026
Percentage of epochs: 0.15
lambda_p: 0.18533319990813935
Percentage of epochs: 0.16
lambda_p: 0.197375320224904
Percentage of epochs: 0.17
lambda_p: 0.2093581694280

In [115]:
input_shape = (64,64,2)

inputs = tf.keras.Input(shape=input_shape)
x = tf.keras.layers.Flatten()(inputs)
x = tf.keras.layers.Dense(units=1024, activation='relu')(x)
x = tf.keras.layers.Dense(units=1024, activation='relu')(x)
x = tf.keras.layers.Dense(units=2, activation=None)(x)    
output = tf.keras.layers.Softmax()(x)
model = tf.keras.Model(inputs = inputs, outputs = [output, x], name = 'Domain_Regressor_FullyConnected')
model.summary()

Model: "Domain_Regressor_FullyConnected"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_121 (InputLayer)      [(None, 64, 64, 2)]       0         
                                                                 
 flatten_50 (Flatten)        (None, 8192)              0         
                                                                 
 dense_189 (Dense)           (None, 1024)              8389632   
                                                                 
 dense_190 (Dense)           (None, 1024)              1049600   
                                                                 
 dense_191 (Dense)           (None, 2)                 2050      
                                                                 
 softmax_55 (Softmax)        (None, 2)                 0         
                                                                 
Total params: 9,441,282
Trainable p

In [116]:
D_out_shape = (1,2,3)

print(len(D_out_shape))

3
